In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import animation
from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib import cm
import matplotlib as mpl
from Tomato3_dataStructure import Trial, Subject, Experiment
from helper import angle_overtake, lateral_overtake, overtake_rate, max_freewalk_spd, average_freewalk_spd


In [2]:
'''import trials'''
Hz = 90
exp = Experiment()
for i in range(1,13):
    exp.subjects[i] = Subject(i)
    if i%2 == 0:
        exp.subjects[i].leader = 'avatar'
    else:
        exp.subjects[i].leader = 'pole'
    
input_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Tomato3_rawData\Tomato3_input'))
output_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Tomato3_rawData\Tomato3_output'))

for output_file in os.listdir(output_dir):
    output_file_path = os.path.join(output_dir, output_file)
    # import experimental data
    if  'Tomato3_subj' in output_file and '.csv' in output_file:
        with open(output_file_path, 'r') as f:
            df = pd.read_csv(f, header=None)
            lpos = np.array(df.iloc[:, [0,2,1]])
            fpos = np.array(df.iloc[:, [3,5,4]])
            fori = np.array(df.iloc[:, 6:9])
            tstamps = np.array(df.iloc[:, 9])
            leader_model = np.array(df.iloc[:, 10])
            subject_id = int(output_file[12:14])
            trial_id = int(output_file[20:23])
            v0 = float(output_file[27:30])
            if output_file[-5] == 'e':
                leader = 'pole'
            elif output_file[-5] == 'r':
                leader = 'avatar'
            leader_onset = None
            
            exp.subjects[subject_id].trials[trial_id] = Trial(subject_id, trial_id, lpos, fpos, fori, \
                                                                tstamps, v0, leader, leader_onset, leader_model)           
    # import IPD and gender data
    elif 'IPD' in output_file:    
        with open(output_file_path, 'r') as f:
            subject_id = int(output_file[12:14])
            exp.subjects[subject_id].gender == output_file[19]
            i = output_file.find('txt')
            exp.subjects[subject_id].IPD == float(output_file[20:i-1])
    # import freewalk data
    elif 'freewalk' in output_file:
        with open(output_file_path, 'r') as f:
            df = pd.read_csv(f, header=None)
            subject_id = int(output_file[21:23])
            session = int(output_file[-14])
            trial_id = int(output_file[-7:-4])
            v0 = 0
            leader = leader_onset = leader_model= None
            fpos = np.array(df.iloc[:,[0,2,1]])
            lpos = np.tile([0,0,0], (len(fpos), 1))
            fori = np.array(df.iloc[:,3:6])
            tstamps = np.array(df.iloc[:,-1])
            if session == 1:
                exp.subjects[subject_id].freewalk[trial_id] = Trial(subject_id, trial_id, lpos, fpos, fori, \
                                                                    tstamps, v0, leader, leader_onset, leader_model)
            else:
                trial_id += 4
                exp.subjects[subject_id].freewalk[trial_id] = Trial(subject_id, trial_id, lpos, fpos, fori, \
                                                                    tstamps, v0, leader, leader_onset, leader_model)
# import inputs
for input_file in os.listdir(input_dir):
    if 'Tomato3_subject' in input_file:
        subject_id = int(input_file[-6:-4])
        if subject_id in exp.subjects and exp.subjects[subject_id].trials != {}:
            # read experimental trials
            with open(os.path.join(input_dir, input_file), 'r') as f:
                df = pd.read_csv(f)
                for i in range(len(df)):
                    trial_id = df.iloc[i,0]
                    leader_onset = df.iloc[i,4]
                    exp.subjects[subject_id].trials[trial_id].leader_onset = leader_onset
 

# with open(filename, 'r') as file:
#     rows = file.read().split('\n')[:-1]
#     cells = [row.split(',') for row in rows]  
#     cells = np.array([[float(s) for s in row] for row in cells])
#     lpos = cells[:,[0,2,1]]
#     fpos = cells[:,[3,5,4]]
#     fori = cells[:,6:9]
#     tstamps = cells[:,-1]



In [18]:
'''debug'''
%matplotlib qt
subject = 12
trial = 38
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        if i == subject and j == trial:
            print((t.get_positions('l')[540:720,1] - t.get_positions('f')[540:720,1])/10)


[0.17506318 0.17499085 0.17491986 0.17485023 0.17478193 0.17471498
 0.17464935 0.17458505 0.17452207 0.1744604  0.17440002 0.17434092
 0.1742831  0.17422654 0.17417121 0.17411712 0.17406423 0.17401254
 0.17396202 0.17391265 0.17386442 0.17381729 0.17377126 0.17372628
 0.17368235 0.17363944 0.17359752 0.17355656 0.17351655 0.17347744
 0.17343923 0.17340187 0.17336535 0.17332963 0.17329469 0.1732605
 0.17322703 0.17319425 0.17316213 0.17313065 0.17309978 0.17306949
 0.17303975 0.17301053 0.17298181 0.17295357 0.17292576 0.17289837
 0.17287138 0.17284475 0.17281846 0.17279248 0.1727668  0.17274139
 0.17271622 0.17269128 0.17266653 0.17264197 0.17261757 0.17259331
 0.17256916 0.17254513 0.17252117 0.17249728 0.17247344 0.17244964
 0.17242586 0.17240208 0.17237829 0.17235449 0.17233064 0.17230675
 0.17228281 0.17225879 0.1722347  0.17221052 0.17218624 0.17216186
 0.17213736 0.17211275 0.172088   0.17206313 0.17203811 0.17201295
 0.17198763 0.17196216 0.17193653 0.17191074 0.17188477 0.17185

In [17]:
'''Plot positions, speed, or acceleration of freewalk trials'''
%matplotlib qt
subject = 3
for i, t in exp.subjects[subject].freewalk.items():
    t.plot_speeds(distance=False)
#     t.plot_positions('f')
#     t.plot_accelerations()


In [3]:
'''plot positions or speed of experimental trials'''
%matplotlib qt
subject = [12]
trials = [15]
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        if i in subject and j in trials:
            if t.v0 == 1.3:
                fpos = t.get_positions('f')
                fspd = t.get_speeds('f')
                label = 'overtake' if lateral_overtake(fpos, fspd, t.v0, 0.3) else ''
                print(round(max(abs(fpos[:,0])), 3))
                t.plot_positions(accelerations=False, links=True, is_filtered=True)
                t.plot_speeds()
                t.plot_accelerations()
                plt.text(0, 0, label)

0.27


In [7]:
'''animation'''
# compare trial 6 (catch up) and 5 (let it go)
%matplotlib qt
subject = 12
trial = 38
exp.subjects[subject].trials[trial].play_trial(save=False, is_filtered=False)

In [3]:
'''lateral deviation in freewalk trials'''
max_deviations = {}
for i, s in exp.subjects.items():
    max_deviation = 0
    for j, t in s.freewalk.items():
        _max = max(abs(t.get_positions('f')[:, 0]))
        if _max > max_deviation:
            max_deviation = _max
    max_deviations[i] = round(max_deviation, 3)
print(max_deviations)

{1: 0.102, 2: 0.097, 3: 0.207, 4: 0.205, 5: 0.281, 6: 0.144, 7: 0.121, 8: 0.148, 9: 0.394, 10: 0.304, 11: 0.128, 12: 0.12}


In [5]:
'''
    Overtake rate by v0 by subject (12 plots). Overtaking is defined by angle 
    and lateral deviation criteria.
'''
%matplotlib qt
for i, s in exp.subjects.items():
    ot_angle = {0.8:0, 0.9:0, 1.0:0, 1.1:0, 1.2:0, 1.3:0}
    ot_lateral = {0.8:0, 0.9:0, 1.0:0, 1.1:0, 1.2:0, 1.3:0}

    for j, t in s.trials.items():
        lpos = t.get_positions('l')
        fpos = t.get_positions('f')
        fspd = t.get_speeds('f')
        if angle_overtake(lpos, fpos, 55):
            ot_angle[t.v0] += 0.1
        if lateral_overtake(fpos, fspd, t.v0, 0.3):
            ot_lateral[t.v0] += 0.1
    fig = plt.figure()
    ax = plt.axes(xlim=(0.7, 1.4), ylim=(-0.1, 1.1))
    ax.plot(list(ot_angle.keys()),list(ot_angle.values()))
    ax.plot(list(ot_lateral.keys()),list(ot_lateral.values()))
    plt.title('subject ' + str(i))

In [14]:
'''
    Overtake rate by speed difference by subject (12 plots). Overtaking is 
    defined by the angle y axis and the line connecting follower and leader 
    greater than 55 degree. 55 is half of field of view in Odyssey HMD.
'''
threshold = np.cos(55 * np.pi / 180)
for i, s in exp.subjects.items():
    dvs = []
    for j, t in s.trials.items():
        lpos = t.get_positions('l')[-1, 0:2]
        fpos = t.get_positions('f')[-1, 0:2]
        vec1 = lpos - fpos
        vec0 = [0, 1]
        cos = np.dot(vec0, vec1)/np.linalg.norm(vec1)
        if cos <= threshold:
            fspd = t.get_speeds('f')[t.f1]
            dvs.append(fspd-t.v0)
    fig = plt.figure()
    ax = plt.axes()
    ax.hist(dvs, 5)

In [3]:
'''
    Overtake rate by v0/fspd_max or fspd_max - v0 by subject (12 plots). 
    Overtaking is defined by lateral deviation and speed criteria.
'''
%matplotlib qt
##########control panel###############
x_axis = 'ratio'
p_fspd_type = 'max' # 'max' or 'mean'
window = 2 # time window used for compute the mean speed
######################################
for i, s in exp.subjects.items():

    # get speed scaler
    if p_fspd_type == 'max':
        p_fspd = max_freewalk_spd(s)
    elif p_fspd_type == 'mean':
        p_fspd = average_freewalk_spd(s, window)
    
    # compute overtake rate
    rate = overtake_rate(s, 0.3)
    
    # plot overtake rate by scaled speed
    fig = plt.figure()
    plt.title('subject ' + str(i))
    if x_axis == 'ratio':
        ax = plt.axes(xlim=(0.4, 1), ylim=(-0.1, 1.1))
        ax.plot([x/10.0/p_fspd for x in range(8, 14)],list(rate.values()))
        ax.plot([0.4, 1],[0.5, 0.5])
    elif x_axis == 'diff':
        ax = plt.axes(xlim=(0, 0.5), ylim=(-0.1, 1.1))
        ax.plot([p_fspd - x/10.0 for x in range(8, 14)],list(rate.values()))
        ax.plot([0, 0.5],[0.5, 0.5])
        
    

In [4]:
'''Compare Overtake rate on pole and avatar'''
%matplotlib qt
fig = plt.figure()
ax = plt.axes(xlim=(0.4, 1), ylim=(-0.1, 1.1))
ax.plot([0.4, 1],[0.5, 0.5], 'k')
for i, s in exp.subjects.items():
    
    # compute the speed scaler
    max_fspd = max_freewalk_spd(s)
    
    # compute overtake rate
    count = overtake_rate(s, 0.3)
    
    # plot overtake rate by scaled speed
    if s.leader == 'pole':
        pole = ax.plot([x/10.0/max_fspd for x in range(8, 14)],list(count.values()), 'b')
    elif s.leader == 'avatar':
        avatar = ax.plot([x/10.0/max_fspd for x in range(8, 14)],list(count.values()), 'r')
ax.legend((pole[0],avatar[0]), ('pole', 'avatar'))